### Dataset Creation and Data Source Augmentation

Create a new column which maps the three digit zip code to the geographic center of all five digit codes that start with the three digit zip.  New column should be titled "zip_geo_center".

In [1]:
import pandas as pd
from uszipcode import SearchEngine
from uszipcode import SimpleZipcode

In [2]:
acq_df = pd.read_csv("D:/Machine_Learning_in_Finance/2010Q1/Acquisition_2010Q1.txt")
zipcode = pd.read_csv("G:/us-zip-code-latitude-and-longitude.csv")

In [3]:
len(acq_df.columns[0])
print(acq_df.columns)

Index(['100010079393|C|WELLS FARGO BANK', ' N.A.|4.875|284000|360|01/2010|03/2010|80|80|1|32|773|Y|P|PU|1|P|TX|787||FRM|||N'], dtype='object')


In [4]:
NAMES = {
    0: "loan_id",
    2: "seller_name",
    3: "original_interest_rate",
    8: "original_loan_to_value",
    11: "debt_to_income",
    12: "credit_score",
    18: "property_state",
    19: "zip_code"
}
col_names = [
    NAMES.get(idx, f"{idx}") for idx, _ in enumerate(range(25))
]
acq_df = pd.read_csv(
    "G:/Stevens_Lecture/2019_Fall/FE690_Machine_Learning_in_Finance/HW2/2010Q1/Acquisition_2010Q1.txt",
    header=None,
    names=col_names,
    sep="|"
)

In [5]:
print(acq_df.columns)

Index(['loan_id', '1', 'seller_name', 'original_interest_rate', '4', '5', '6',
       '7', 'original_loan_to_value', '9', '10', 'debt_to_income',
       'credit_score', '13', '14', '15', '16', '17', 'property_state',
       'zip_code', '20', '21', '22', '23', '24'],
      dtype='object')


In [6]:
# converting zip code to categorical data
zipcode['Zip'] = zipcode['Zip'].map(lambda x: str(x).zfill(5))
zipcode["zip_3"] = zipcode['Zip'].map(lambda x: x[:3])

In [7]:
acq_df['zip_code'] = acq_df['zip_code'].map(lambda x: str(x).zfill(3)) 

In [8]:
search = SearchEngine(simple_zipcode = True)

def geo_ctr(x):
    result = zipcode[zipcode['zip_3'] == str(x)][['Latitude', 'Longitude']].mean()
    ctr = search.by_coordinates(result[0], result[1], returns = 1)
    if len(ctr) == 0:
        return zipcode[zipcode['zip_3'] == str(x)].iloc[0, 0]
    return ctr[0].zipcode

In [10]:
acq_df['zip_geo_center'] = acq_df['zip_code'].map(geo_ctr)

In [16]:
acq_df.head(10)

,loan_id,1,seller_name,original_interest_rate,4,5,6,7,original_loan_to_value,9,...,16,17,property_state,zip_code,20,21,22,23,24,zip_geo_center
0,100010079393,C,"WELLS FARGO BANK, N.A.",4.875,284000,360,01/2010,03/2010,80,80.0,...,1,P,TX,787,NaN,FRM,NaN,NaN,N,78701
1,100013622306,R,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",4.750,87000,180,12/2009,02/2010,63,63.0,...,1,P,CA,932,NaN,FRM,785.0,NaN,N,93219
2,100019943199,R,OTHER,5.000,417000,360,11/2009,01/2010,43,43.0,...,1,S,FL,342,NaN,FRM,808.0,NaN,N,34232
3,100022098429,R,OTHER,5.250,461000,360,01/2010,03/2010,61,61.0,...,2,P,NY,112,NaN,FRM,NaN,NaN,N,11203
4,100023088745,R,"WELLS FARGO BANK, N.A.",5.250,100000,360,11/2009,01/2010,80,80.0,...,1,P,OH,446,NaN,FRM,NaN,NaN,N,44612
5,100027393681,B,"CITIMORTGAGE, INC.",4.250,274000,120,01/2010,03/2010,65,82.0,...,1,P,CA,913,NaN,FRM,NaN,NaN,N,91324
6,100027760956,R,OTHER,5.250,227000,360,01/2010,03/2010,55,55.0,...,1,P,CA,934,NaN,FRM,799.0,NaN,N,93401
7,100031032487,R,"BANK OF AMERICA, N.A.",5.125,730000,360,03/2010,04/2010,63,63.0,...,1,P,MD,208,NaN,FRM,NaN,NaN,N,20850
8,100032715932,R,SUNTRUST MORTGAGE INC.,5.375,96000,360,12/2009,01/2010,75,75.0,...,3,I,LA,711,NaN,FRM,815.0,NaN,N,71103
9,100033339771,R,OTHER,5.250,300000,360,02/2010,04/2010,74,74.0,...,1,P,ID,836,NaN,FRM,NaN,NaN,N,83629
